In [10]:
import os
def _set_env_from_file(var: str, file_path: str = "openai_key.txt"):
    """
    Reads an API key from a specified file and sets it as an environment variable.
    """
    if not os.environ.get(var):
        try:
            # The 'with open' statement ensures the file is closed automatically
            with open(file_path, 'r') as f:
                # Read the first line and strip any leading/trailing whitespace
                key = f.readline().strip()

            if key:
                os.environ[var] = key
                print(f"Successfully loaded {var} from {file_path}")
            else:
                print(f"Warning: {file_path} is empty.")

        except FileNotFoundError:
            print(f"Error: Key file not found at {file_path}. Please create the file.")

# --- Execution ---
# Set the environment variable OPENAI_API_KEY from the file
_set_env_from_file('OPENAI_API_KEY',file_path="../../keys/openai.txt")
_set_env_from_file('TAVILY_API_KEY',file_path="../../keys/tavily")

Successfully loaded TAVILY_API_KEY from ../../keys/tavily


In [2]:
MODEL="openai:gpt-4.1"

In [ ]:
# %pip install langchain-openai

In [8]:
from typing import Annotated
from langchain_tavily import TavilySearch
from langchain_core.messages import BaseMessage
from typing_extensions import TypedDict
from langgraph.graph import StateGraph,START,END,MessagesState
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
from langchain.chat_models import init_chat_model

In [11]:
graph_builder = StateGraph(MessagesState)
llm = init_chat_model(model=MODEL)
web_search_tool = TavilySearch(max_results=2)
tools=[web_search_tool]

llm_with_tools = llm.bind_tools(tools=tools)

def chatbot(state:MessagesState):
    print(f"Starting chatbot with {state}")
    print(f"chatbot invoking {state['messages']}")
    return {"messages" : [llm_with_tools.invoke(state['messages'])]}

In [12]:
tool_node = ToolNode(tools=tools)

graph_builder.add_node("chatbot",chatbot)
graph_builder.add_node("tools",tool_node)
graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition
)

graph_builder.add_edge("tools","chatbot")
graph_builder.add_edge(START,"chatbot")
graph = graph_builder.compile()

In [14]:
print(graph.get_graph().print_ascii());

        +-----------+         
        | __start__ |         
        +-----------+         
               *              
               *              
               *              
          +---------+         
          | chatbot |         
          +---------+         
          .         .         
        ..           ..       
       .               .      
+---------+         +-------+ 
| __end__ |         | tools | 
+---------+         +-------+ 
None


In [15]:
output = graph.invoke({"messages" : ["Explain what you understand with confidence intervals in statistics"]})

Starting chatbot with {'messages': [HumanMessage(content='Explain what you understand with confidence intervals in statistics', additional_kwargs={}, response_metadata={}, id='e0094448-bda4-454f-8beb-7c946bd74fbe')]}
chatbot invoking [HumanMessage(content='Explain what you understand with confidence intervals in statistics', additional_kwargs={}, response_metadata={}, id='e0094448-bda4-454f-8beb-7c946bd74fbe')]


In [16]:
output

{'messages': [HumanMessage(content='Explain what you understand with confidence intervals in statistics', additional_kwargs={}, response_metadata={}, id='e0094448-bda4-454f-8beb-7c946bd74fbe'),
  AIMessage(content='A **confidence interval** in statistics is a range of values, derived from sample data, that is likely to contain the true value of an unknown population parameter (such as the mean or proportion). It provides an estimate of uncertainty around the sample statistic.\n\n### Key Points:\n- **Interval Estimate:** Unlike a single-point estimate (e.g., "the average height is 170 cm"), a confidence interval gives a range (e.g., "the true average height is likely between 168 cm and 172 cm").\n- **Confidence Level:** The confidence interval is associated with a confidence level, usually expressed as a percentage (commonly 90%, 95%, or 99%). A 95% confidence interval means that if we were to take many samples and compute intervals the same way, about 95% of those intervals would conta

In [17]:
from langgraph.checkpoint.memory import MemorySaver
memory = MemorySaver()

In [19]:
graph = graph_builder.compile(checkpointer=memory)
config = {"configurable":{"thread_id":"1"}}
user_input ="Sup i am a llm enthusiast"

events = graph.stream(
    {"messages" : [{
        "role":"user",
        "content":user_input,}]},
    config=config,
    stream_mode="values",
)

for event in events:
    event["messages"][-1].pretty_print()

================================ Human Message =================================

Sup i am a llm enthusiast
Starting chatbot with {'messages': [HumanMessage(content='Sup i am a llm enthusiast', additional_kwargs={}, response_metadata={}, id='8a135ea8-755a-4118-bab2-cd0aa877508c')]}
chatbot invoking [HumanMessage(content='Sup i am a llm enthusiast', additional_kwargs={}, response_metadata={}, id='8a135ea8-755a-4118-bab2-cd0aa877508c')]
================================== Ai Message ==================================

Hey! That’s awesome—large language models (LLMs) are such a fascinating field with so much happening right now. Are you working on any projects, research, or just exploring the tech? Let me know if you want to discuss models, prompting, fine-tuning, or anything else!


In [20]:
user_input ="What is my enthusiasm?"

events = graph.stream(
    {"messages": [{"role":"user","content":user_input}]},
    config=config,
    stream_mode="values",
)

for event in events:
    event["messages"][-1].pretty_print()

================================ Human Message =================================

What is my enthusiasm?
Starting chatbot with {'messages': [HumanMessage(content='Sup i am a llm enthusiast', additional_kwargs={}, response_metadata={}, id='8a135ea8-755a-4118-bab2-cd0aa877508c'), AIMessage(content='Hey! That’s awesome—large language models (LLMs) are such a fascinating field with so much happening right now. Are you working on any projects, research, or just exploring the tech? Let me know if you want to discuss models, prompting, fine-tuning, or anything else!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 61, 'prompt_tokens': 1275, 'total_tokens': 1336, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-2025-04-14', 'system_fingerprint': 'fp_e24a1fec47', 'id': 'ch